In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(
    name = "quick-starts-ws-146112",
    resource_group = 'aml-quickstarts-146112',
    subscription_id = '9a7511b8-150f-4a58-8528-3e7d50216c31'
)
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-146112
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-146112


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
ccName = "cpucluster"

# Verify that cluster does not exist already
try:
    cc = ComputeTarget(workspace = ws, name = ccName)
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        max_nodes = 4
    )
    
    cc = ComputeTarget.create(ws, ccName, compute_config)

cc.wait_for_completion(show_output = True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(.1, 10),
    '--max_iter': choice(range(1, 1000))
})### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval = 2,
    slack_factor = .1
)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory = '.',
    compute_target = cc,
    entry_script = 'train.py'
)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator = est,
    hyperparameter_sampling = ps,
    policy = policy,
    primary_metric_name = 'Accuracy',
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 20,
    max_concurrent_runs = 4,
    max_duration_minutes = 60
)### YOUR CODE HERE ###

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdr = exp.submit(hyperdrive_config)

RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
hdr.wait_for_completion(show_output = True)

hdr_bestRun = hdr.get_best_run_by_primary_metric()

#retrive the best run configs and train a model using the configs
paramVals = hdr_bestRun.get_details()['runDefinition']['arguments']

#view vals
paramVals

RunId: HD_8776c3bf-79f5-4338-a227-f16e6beec90a
Web View: https://ml.azure.com/runs/HD_8776c3bf-79f5-4338-a227-f16e6beec90a?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-146112/workspaces/quick-starts-ws-146112&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-05-29T17:47:24.907116][API][INFO]Experiment created<END>\n""<START>[2021-05-29T17:47:25.371965][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-05-29T17:47:25.578392][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-05-29T17:47:55.1899005Z][SCHEDULER][INFO]Scheduling job, id='HD_8776c3bf-79f5-4338-a227-f16e6beec90a_1'<END><START>[2021-05-29T17:47:55.1877086Z][SCHEDULER][INFO]Scheduling job, id='HD_8776c3bf-79f5-4338-a227-f16e6beec90a_0'<END><START>[2021-05-29T17:47:55.2378391Z][SCHEDULER][INFO]Scheduling job, id='HD_8776c3bf-79f5-4

['--C', '5.771294653995883', '--max_iter', '957']

In [7]:
from train import trainMdl

hdr_mdl = trainMdl(float(paramVals[1]), int(paramVals[3]))    
    
#save the model
joblib.dump(hdr_mdl, 'hdr_mdl.joblib')

['hdr_mdl.joblib']

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
from train import getData

ds = getData()

In [10]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [19]:
#view data
print(x.head())

y.head()

   age  marital  default  housing  loan  month  day_of_week  duration  \
0   57        1        0        0     1      5            1       371   
1   55        1        0        1     0      5            4       285   
2   33        1        0        0     0      5            5        52   
3   36        1        0        0     0      6            5       355   
4   27        1        0        1     0      7            5       189   

   campaign  pdays  ...  contact_cellular  contact_telephone  \
0         1    999  ...                 1                  0   
1         2    999  ...                 0                  1   
2         1    999  ...                 1                  0   
3         4    999  ...                 0                  1   
4         2    999  ...                 1                  0   

   education_basic.4y  education_basic.6y  education_basic.9y  \
0                   0                   0                   0   
1                   0                   0     

0    0
1    0
2    0
3    0
4    0
Name: y, dtype: int64

In [24]:
type(y)

pandas.core.series.Series

In [25]:
#combine
import pandas as pd

td = x.merge(y.to_frame(), left_index = True, right_index = True)

td.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [26]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

from azureml.core.datastore import Datastore

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task = 'classification',
    primary_metric = 'accuracy',
    training_data = TabularDatasetFactory.register_pandas_dataframe(
        td,
        name = 'bankmarketing_clean',
        target = Datastore(ws)
    ),
    label_column_name = 'y',
    n_cross_validations = 10,
    max_concurrent_iterations = 4,
    compute_target = cc
)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/74d26f57-d809-4a88-9e1c-e843f2c512e3/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [27]:
# Submit your automl run

### YOUR CODE HERE ###
amlr = exp.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpucluster with default configuration
Running on remote compute: cpucluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5bb89b6c-a6f8-425a-935f-1ee00bd13f85,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

        49   SparseNormalizer XGBoostClassifier             0:04:27          nan    0.9165
ERROR: {
    "additional_properties": {},
    "error": {
        "additional_properties": {
            "additionalInfo": null,
            "debugInfo": null
        },
        "code": "UserError",
        "severity": null,
        "message": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_format": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_parameters": {},
        "reference_code": null,
        "details_uri": null,
        "target": null,
        "details": [],
        "inner_error": {
            "additional_properties": {},
            "code": "ResourceExhausted",
            "inner_error": {
                "additional_properties": {},
                "code": "Timeout",
                "inner_error": {
                    "additional_properties": {},
                    "code": "Exper

In [28]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
RunDetails(amlr).show()

amlr.wait_for_completion(show_output = True)

amlr_bestRun, amlr_bestMdl = amlr.get_output()

joblib.dump(amlr_bestMdl, 'amlr_mdl.joblib')

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5bb89b6c-a6f8-425a-935f-1ee00bd13f85,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|3692                             |1                                |32950                                 |
+---------------------------------+----------

        50   MaxAbsScaler LightGBM                          0:00:51       0.8880    0.9165
        51   StandardScalerWrapper XGBoostClassifier        0:01:39       0.9089    0.9165
        52   SparseNormalizer XGBoostClassifier             0:01:49       0.9157    0.9165
        53   SparseNormalizer XGBoostClassifier             0:00:53          nan    0.9165
ERROR: {
    "additional_properties": {},
    "error": {
        "additional_properties": {
            "additionalInfo": null,
            "debugInfo": null
        },
        "code": "UserError",
        "severity": null,
        "message": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_format": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_parameters": {},
        "reference_code": null,
        "details_uri": null,
        "target": null,
        "details": [],
        "inner_error": {
            "additional_properties

['amlr_mdl.joblib']